In [1]:
import pickle

import numpy as np
import tensorflow as tf
from tf_smpl.batch_smpl import SMPL


In [4]:
print(tf.__version__)

In [3]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True

sess = tf.InteractiveSession(config=config)

In [5]:
with open('mpi/data/mano/MANO_RIGHT_py3.pkl', 'rb') as f:
    mano_data = pickle.load(f, encoding='latin1')

In [11]:
batch_size = mano_data['hands_coeffs'].shape[0]

### Initializing SMPL model

In [7]:
mano = SMPL(mano_data)

In [8]:
hands_components = tf.Variable(mano_data['hands_coeffs'], dtype=tf.float32)
hands_coeffs = tf.Variable(mano_data['hands_components'], dtype=tf.float32)

hands_poses = tf.matmul(hands_components, hands_coeffs)

In [12]:
# cam = N x 3, pose N x self.num_theta, shape: N x 10
rot = tf.Variable([[0, 3.14/2, 0]], expected_shape=[1,3])
cams = tf.tile(rot, [1554, 1])

#sess.run(tf.global_variables_initializer())
#print(sess.run(cams))

poses = tf.Variable(tf.concat([cams, mano_data['hands_mean'] + hands_poses], axis=1))
shapes = tf.Variable(tf.zeros([batch_size, 10]))

2


In [15]:
sess.run(tf.global_variables_initializer())

In [13]:
verts, joints, Rs = mano(shapes, poses, get_skin=True)

In [16]:
verts = sess.run(verts)

In [4]:

assert tf.test.is_gpu_available()

AssertionError: 